<a href="https://colab.research.google.com/github/GraceW18/image-classification-model-CNN/blob/main/CNNmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from collections import defaultdict

# Load and reshape data
train_data = pd.read_csv('sign_mnist_13bal_train.csv')
test_data = pd.read_csv('sign_mnist_13bal_test.csv')
X_train = train_data.drop('class', axis=1).values / 255.0
y_train = train_data['class'].values
X_test = test_data.drop('class', axis=1).values / 255.0
y_test = test_data['class'].values

# Clean label values (remove/mapping for invalid ones if needed)
y_train[y_train == 10] = 9
y_test[y_test == 10] = 9

# Reshape features from flat to 28x28x1 for images
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
num_classes = 10

datagen = ImageDataGenerator(
    rotation_range=8,
    width_shift_range=0.08,
    height_shift_range=0.08,
    shear_range=0.05,
    zoom_range=0.08,
    horizontal_flip=False,
    vertical_flip=False
)
datagen.fit(X_train)

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,
                              patience=3, min_lr=1e-5, verbose=1)

# Early stopping for generalization
early_stop = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# Build improved CNN model
model = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=0.005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Generate augmented data batches and fit the model
model.fit(datagen.flow(X_train, y_train, batch_size=16), epochs=150,
          validation_data=(X_test, y_test), callbacks=[reduce_lr, early_stop], verbose=2)

# Get predictions for training and test sets
y_pred_train = np.argmax(model.predict(X_train), axis=1)
y_pred = np.argmax(model.predict(X_test), axis=1)

# Print network summary and architecture layers
print(f"Training set size: {len(y_train)}")
print(f"Layer sizes: Conv2D(32)->Conv2D(64)->Dense(128)->Dense({num_classes})")

# Per-class and overall accuracy calculations
correct_counts = defaultdict(int)
total_counts = defaultdict(int)
overall_correct = 0
for true, pred in zip(y_test, y_pred):
    total_counts[true] += 1
    if true == pred:
        correct_counts[true] += 1
        overall_correct += 1

total_counts_training = 0
correct_counts_training = 0
for true, pred in zip(y_train, y_pred_train):
    total_counts_training += 1
    if true == pred:
        correct_counts_training += 1

for class_id in sorted(total_counts.keys()):
    accuracy = correct_counts[class_id] / total_counts[class_id] * 100
    print(f"Accuracy for class {class_id}: {accuracy:3.0f}%")
print("----------")
overall_accuracy = overall_correct / len(y_test) * 100
print(f"Overall Test Accuracy: {overall_accuracy:3.1f}%")
overall_training_accuracy = correct_counts_training / total_counts_training * 100
print(f"Overall Training Accuracy: {overall_training_accuracy:3.1f}%")

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
for i, row in enumerate(conf_matrix):
    print(f"Class {i}:", " ".join(f"{num:5d}" for num in row))

Epoch 1/150


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 - 5s - 505ms/step - accuracy: 0.1462 - loss: 5.0435 - val_accuracy: 0.0923 - val_loss: 2.5133 - learning_rate: 0.0050
Epoch 2/150
9/9 - 1s - 57ms/step - accuracy: 0.4077 - loss: 3.1784 - val_accuracy: 0.1077 - val_loss: 4.4185 - learning_rate: 0.0050
Epoch 3/150
9/9 - 1s - 62ms/step - accuracy: 0.5077 - loss: 2.3279 - val_accuracy: 0.1462 - val_loss: 2.8140 - learning_rate: 0.0050
Epoch 4/150
9/9 - 1s - 63ms/step - accuracy: 0.5308 - loss: 2.0158 - val_accuracy: 0.2077 - val_loss: 3.0833 - learning_rate: 0.0050
Epoch 5/150
9/9 - 1s - 63ms/step - accuracy: 0.5538 - loss: 1.8829 - val_accuracy: 0.2385 - val_loss: 2.4543 - learning_rate: 0.0050
Epoch 6/150
9/9 - 1s - 60ms/step - accuracy: 0.6462 - loss: 1.7362 - val_accuracy: 0.1846 - val_loss: 3.3106 - learning_rate: 0.0050
Epoch 7/150
9/9 - 1s - 70ms/step - accuracy: 0.6385 - loss: 1.5088 - val_accuracy: 0.2769 - val_loss: 2.8391 - learning_rate: 0.0050
Epoch 8/150
9/9 - 1s - 62ms/step - accuracy: 0.6385 - loss: 1.3727 - val_accurac